# COGS 189 Analysis

In [1]:
import numpy as np   
import matplotlib.pyplot as plt    
import pandas as pd
import mne

# Read data with added stim channel
See find_peaks_stim_channel.ipynb for generating the stim channel from aux data.

In [2]:
data = pd.read_csv("exp0_sub0_8ch_1stim.csv")
data.head()

,Unnamed: 0,EXG0,EXG1,EXG2,EXG3,EXG4,EXG5,EXG6,EXG7,Analog0,Analog1,Analog2,Timestamp,stim
0,0,12584.725032,15524.024132,-2317.138292,17950.261289,14667.952319,7234.834997,18205.361748,19735.338656,421.0,621.0,511.0,1.677707e+09,0
1,1,12595.587980,15529.410902,-2316.959478,17958.352620,14691.756927,7232.622174,18216.045882,19740.792482,402.0,516.0,511.0,1.677707e+09,0
2,2,12579.830000,15523.957077,-2315.327801,17950.641269,14672.221502,7231.638698,18202.008987,19733.751682,387.0,445.0,511.0,1.677707e+09,0
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,445.0,768.0,511.0,1.677707e+09,0
4,4,12535.305325,15492.910503,-2324.290851,17940.381818,14577.673623,7226.006058,18228.495804,19714.976217,423.0,622.0,511.0,1.677707e+09,0


# Import Channel Map
(this was just written out as a txt file during data collection)

In [3]:
chan_map_path = r'../data/exp0/OpenBCISession_2023-03-01_13-30-42/chan_map.txt'
chan_map = pd.read_csv(chan_map_path,
                      delimiter='\t')
chan_map

,Cyton,004
0,REF,CPz
1,GND,AFz
2,1,Fz
3,2,Cz
4,3,Pz
5,4,Fp1
6,5,F7
7,6,T6
8,7,T4
9,8,T3


# Make an MNE Info Object

In [4]:
FS_EEG = 250.0

col_names = data.keys()
print('#Channels:', len(col_names))

# Container to populate with MNE channel types
ch_names = ['']*len(ch_names)
ch_types = ['']*len(ch_names)

ch_ndx = 0
for col in col_names:
    # print(ch_ndx, col)
    if 'EXG' in col:
        ch_names[ch_ndx] = chan_map.iloc[ch_ndx+1, 1]
        print(ch_ndx, ch_names[ch_ndx])
        ch_types[ch_ndx] = 'eeg'
    elif 'stim' in col:
        ch_names[ch_ndx] = col
        ch_types[ch_ndx] = 'stim'
    else: # Analog / Timestamp
        ch_names[ch_ndx] = col
        ch_types[ch_ndx] = 'misc'
    ch_ndx += 1

info = mne.create_info(
    ch_names=ch_names, 
    sfreq=FS_EEG, 
    ch_types=ch_types
)

# Apply 10-20 montage for these channels
info.set_montage('standard_1020', match_case=False)

#Channels: 14


NameError: name 'ch_names' is not defined

# Import the data into MNE

In [ ]:
# Need to transpose the data
raw = mne.io.RawArray(data.T, info)

raw

# Try a PSD plot of raw data
Watch out for 60 Hz!

In [ ]:
scalings = {'eeg': 100, 'bio': 1, 'time': 1}

raw.plot(n_channels=8, scalings=scalings, title='test',
         show=True, block=True)


raw.compute_psd().plot(average = False)

# Bandpass Filter
https://mne.tools/dev/auto_tutorials/preprocessing/30_filtering_resampling.html

In [ ]:
raw_filtered = raw.copy().filter(
    l_freq = 0.1, 
    h_freq = 30
)

In [ ]:
raw_filtered.compute_psd().plot(average = False)

# Epoch
https://mne.tools/stable/generated/mne.find_events.html

## Trigger Labels

Trigger | Description
:-- | :--
0 | nothing
1 | trial start (loud cue)
2 | loud cue
3 | soft cue
X | auto-assigned by stim wave used

X by stim file used:

```
 'audio/exp0/kick_iso.wav': 10,
 'audio/exp0/kick_mp1.wav': 11,
 'audio/exp0/kick_mp2.wav': 12,
 'audio/exp0/snare_iso.wav': 13,
 'audio/exp0/snare_mp1.wav': 14,
 'audio/exp0/snare_mp2.wav': 15
```

In [ ]:
mne.find_events(raw)